In [1]:
from google.colab import auth
auth.authenticate_user()

import sys
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/Colab Notebooks/VGG/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/VGG


In [0]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [0]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(64,64, kernel_size=3, padding=1)
        self.maxPool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(64,128, kernel_size=3, padding=1)
        #self.bn2 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        
        self.conv5 = nn.Conv2d(128,256, kernel_size=3, padding=1)
        #self.bn3 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256,256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256,256, kernel_size=1, padding=0)
        
        self.conv8 = nn.Conv2d(256,512, kernel_size=3, padding=1)
        #self.bn4 = nn.BatchNorm2d(512)
        
        self.conv9 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512,512, kernel_size=1, padding=0)
        
        self.conv11 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(512,512, kernel_size=1, padding=0)
        
        self.linear1 = nn.Linear(512*7*7, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        out= self.conv1(x)
        out = self.conv2(out)
        out = self.maxPool(out)
        
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxPool(out)
        
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.conv7(out)
        out = self.maxPool(out)
        
        out = self.conv8(out)
        out = self.conv9(out)
        out = self.conv10(out)
        out = self.maxPool(out)
        
        out = self.conv11(out)
        out = self.conv12(out)
        out = self.conv13(out)

        out = self.maxPool(out)
        
        out = out.view(out.size(0),-1)
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)
        
        return out
        
        

In [15]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



dataset_train = CIFAR10(root='../data', train=True, 
            download=True, transform=transform)
dataset_test = CIFAR10(root='../data', train=False, 
             download=True, transform=transform)
train_loader = DataLoader(dataset_train, batch_size=32, 
                        shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, 
                       shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
model=VGG16()
model=model.to(device)

In [0]:
criterio=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
device=torch.device('cuda')
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
NUM_EPOCHS=100

In [0]:
model.train()
for epoch in range(NUM_EPOCHS):
  for imgs, classes in train_loader:
    #print(imgs.shape, classes.shape)
    imgs, classes=imgs.to(device), classes.to(device)
    outputs = model(imgs)

    loss = criterio(outputs, classes)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, preds = torch.max(outputs, 1)
    #print(loss.item())
    #print(preds)

  if epoch%10 == 0:
      print('Epoch {}: Loss = {} Accuracy = {} '.format(epoch+1, loss.item(),torch.sum(preds == classes).item()/32))

Epoch 1: Loss = 602881916928.0 Accuracy = 0.03125 


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

for i, (input_,target) in enumerate(train_loader):
    target = target.to(device)
    input_ = input_.to(device)
    print(input_.shape)
    print(target.shape, input_.shape)

    output = model(input_)
    loss = criterio(output,target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i%20 == 0):
        print("loss in epoch %d , step %d : %f" % (epoch, i,loss.data[0]))

torch.Size([32, 3, 224, 224])
torch.Size([32]) torch.Size([32, 3, 224, 224])
torch.Size([32, 64, 224, 224])
torch.Size([32, 64, 224, 224])
torch.Size([32, 64, 112, 112])
torch.Size([32, 128, 112, 112])
torch.Size([32, 128, 112, 112])
torch.Size([32, 128, 56, 56])
torch.Size([32, 256, 56, 56])
torch.Size([32, 256, 56, 56])
torch.Size([32, 256, 56, 56])
torch.Size([32, 256, 28, 28])
torch.Size([32, 512, 28, 28])
torch.Size([32, 512, 28, 28])
torch.Size([32, 512, 28, 28])
torch.Size([32, 512, 14, 14])
torch.Size([32, 512, 14, 14])
torch.Size([32, 512, 14, 14])
torch.Size([32, 512, 14, 14])
torch.Size([32, 512, 7, 7])
torch.Size([32, 25088])


NameError: ignored